In [1]:
import numpy as np
import datetime
import json
import pylab
import pandas as pd
import matplotlib.pyplot as plt

# Preparacion de datos

## Cargando datos

In [2]:

def leer_data():
    outfile='./data2/consulta.csv'
    data = pd.read_csv(outfile)
    return data

data_original = leer_data()
data_original.head(3)

,client_id,date,año,mes,dia,hora,merchant_departement,merchant_province,merchant_district,mcc,mccg,quantity,amount_sol
0,+++g8j9k+5A=,2016-09-27 01:37:23,2016,9,27,1,LIMA,LIMA,San Juan De Miraflor,5912,15,1,107.0
1,+++g8j9k+5A=,2016-06-24 03:35:00,2016,6,24,3,LIMA,LIMA,Magdalena Del Mar,5912,15,1,58.0
2,+++g8j9k+5A=,2017-04-29 03:15:16,2017,4,29,3,LIMA,LIMA,Miraflores,6300,17,1,153.0


## Preparacion de datos

In [3]:
data = data_original[['client_id','date','año','mes','dia','hora','quantity','amount_sol']]
data.head(3)   

,client_id,date,año,mes,dia,hora,quantity,amount_sol
0,+++g8j9k+5A=,2016-09-27 01:37:23,2016,9,27,1,1,107.0
1,+++g8j9k+5A=,2016-06-24 03:35:00,2016,6,24,3,1,58.0
2,+++g8j9k+5A=,2017-04-29 03:15:16,2017,4,29,3,1,153.0


# Unidad de TXs temporales (U)

In [12]:

file='U'
raw_data='./data2/%s.json' %(file)



## Funciones

In [5]:
# definimos los 4 time_windows que usaremos

def time_window(hora):
    tw = 9999
    if hora >=0:
        tw = 0      # Madrugada
    if hora >=6:
        tw = 1      # Mañana
    if hora >=12:
        tw = 2      # Tarde
    if hora >=18:
        tw = 3      # Noche
    return tw

In [6]:
# Definimos los U 

def procesar_u(user):    
    uid=list(user['client_id'])[0]
    
    # Lista los años en que tiene txs el usuario
    years = set(list(user['año']))
    anni = {year:{} for year in list(years)}
    
    # para cada fila (para cada fecha)
    for dat in  range(0,len(user)):
        año = user.iloc[dat]['año']
        week=datetime.datetime(año,user.iloc[dat]['mes'],user.iloc[dat]['dia']).isocalendar()[1]
        weekday=datetime.datetime(año,user.iloc[dat]['mes'],user.iloc[dat]['dia']).weekday()
        turn = time_window(data.iloc[dat]['hora'])
        
        # Si la semana no existe en el año
        if not(week in anni[año]):
            anni[año][week] = {}
        # Si el turno no existe en la semana y año
        if not (turn in anni[año][week]):
            anni[año][week][turn]=np.array([0]*7)  #CUATRO TURNOS
            
        anni[año][week][turn][weekday]+=user.iloc[dat]['quantity'] # suma cantidades "importancia por compras"
        #anni[año][week][turn][weekday]+=user.iloc[dat]['amount_sol'] # suma montos "importancia por gastos"
            
    return uid,anni

## Procesando U de cada cliente

In [7]:
# Extraemos la lista de clientes sin repetir
clientes =  data.groupby('client_id').client_id.count().index

profiles={}
# Para cada cliente
for cliente in clientes:
    cliente_i= data[data['client_id'] == cliente]
    ## ejecutamos para cada usuario
    results=procesar_u(cliente_i)
    profiles[results[0]]=results[1]

In [8]:
profiles

{'+++g8j9k+5A=': {2016: {39: {0: array([0, 1, 0, 0, 0, 0, 0])},
   25: {0: array([0, 0, 0, 0, 2, 0, 0])},
   22: {2: array([0, 0, 0, 0, 0, 0, 1])},
   26: {2: array([0, 0, 0, 0, 0, 0, 1]), 3: array([0, 0, 1, 0, 0, 0, 0])},
   37: {1: array([0, 0, 0, 0, 0, 1, 0])},
   27: {0: array([0, 0, 0, 1, 0, 0, 0])},
   49: {0: array([0, 0, 0, 0, 1, 0, 0])}},
  2017: {17: {0: array([0, 0, 0, 0, 0, 2, 0])},
   9: {0: array([0, 0, 2, 1, 0, 0, 0])},
   22: {0: array([0, 2, 0, 0, 0, 0, 0])},
   11: {2: array([1, 0, 0, 0, 0, 0, 0]), 1: array([0, 0, 1, 0, 0, 0, 0])},
   5: {0: array([0, 0, 0, 0, 2, 0, 0])},
   12: {3: array([0, 0, 3, 0, 0, 0, 0])},
   13: {0: array([1, 0, 0, 2, 0, 0, 0])},
   21: {3: array([0, 0, 0, 1, 0, 0, 0])}}},
 '++/9crwdWSc=': {2016: {45: {0: array([1, 1, 0, 0, 0, 0, 0]),
    2: array([1, 1, 0, 0, 0, 0, 0]),
    3: array([0, 0, 0, 0, 0, 1, 0])},
   32: {0: array([0, 0, 0, 0, 1, 0, 0])},
   31: {0: array([0, 1, 0, 0, 0, 0, 0]),
    2: array([0, 0, 0, 1, 2, 0, 0]),
    3: array([0, 

In [27]:
individual_footprint="%s.individual_footprint" %(raw_data)

#exporting individual footprint   <customer_id;year;week;profile_id;turn;size;t1... tn >
fw=open(individual_footprint,'w')
fw.write('customer_id,year,week,profile_id,turn,size,d0t0,d1t0,d2t0,d3t0,d4t0,d5t0,d6t0,d0t1,d1t1,d2t1,d3t1,d4t1,d5t1,d6t1,d0t2,d1t2,d2t2,d3t2,d4t2,d5t2,d6t2,d0t3,d1t3,d2t3,d3t3,d4t3,d5t3,d6t3\n')
footprints=0
for uid in profiles:
    profile_id=0
    for year in profiles[uid]:
        for week in profiles[uid][year]:
            temp=np.zeros(28)
            for turn in profiles[uid][year][week]:
                d=profiles[uid][year][week][turn]
                if(turn == 0):
                    for i in range(0,7):
                        temp[i] += d[i]
                if(turn == 1):
                    for i in range(7,14):
                        temp[i] += d[i-7]
                if(turn == 2):
                    for i in range(14,21):
                        temp[i] += d[i-14]
                if(turn == 3):
                    for i in range(21,28):
                        temp[i] += d[i-21]
                
                
            fw.write(''+str(uid)+','+str(year)+','+str(week)+','+str(profile_id)+','+str(turn)+','+str(sum(temp))+','
                     +str(temp[0])+','+str(temp[1])+','+str(temp[2])+','+str(temp[3])+','+str(temp[4])+','+str(temp[5])+','+str(temp[6])+','
                     +str(temp[7])+','+str(temp[8])+','+str(temp[9])+','+str(temp[10])+','+str(temp[11])+','+str(temp[12])+','+str(temp[13])+','
                     +str(temp[14])+','+str(temp[15])+','+str(temp[16])+','+str(temp[17])+','+str(temp[18])+','+str(temp[19])+','+str(temp[20])+','
                     +str(temp[21])+','+str(temp[22])+','+str(temp[23])+','+str(temp[24])+','+str(temp[25])+','+str(temp[26])+','+str(temp[27])
                     +'\n')

            profile_id = profile_id + 1
    footprints+=profile_id
    fw.flush()
fw.close()
print ("number of footprint: "+str(footprints))

number of footprint: 41705


In [21]:
np.zeros(5)

array([0., 0., 0., 0., 0.])

In [23]:
for i in range(0,7):
    print(i)

0
1
2
3
4
5
6


In [24]:
for i in range(7,14):
    print(i)

7
8
9
10
11
12
13


In [ ]:
individual_footprint="%s.individual_footprint" %(raw_data)

#exporting individual footprint   <customer_id;year;week;profile_id;turn;size;t1... tn >
fw=open(individual_footprint,'w')
fw.write('customer_id,year,week,profile_id,turn,size,d0,d1,d2,d3,d4,d5,d6\n')
footprints=0
for uid in profiles:
    profile_id=0
    for year in profiles[uid]:
        for week in profiles[uid][year]:
            for turn in profiles[uid][year][week]:
                d=profiles[uid][year][week][turn]
                
                fw.write(''+str(uid)+','+str(year)+','+str(week)+','+str(profile_id)+','+str(turn)+','
                         +str(sum(d))+','+str(d[0])+','+str(d[1])+','+str(d[2])+','+str(d[3])+
                         ','+str(d[4])+','+str(d[5])+','+str(d[6])+'\n')

            profile_id = profile_id + 1
    footprints+=profile_id
    fw.flush()
fw.close()
print ("number of footprint: "+str(footprints))

# Temporal TXs footprint

In [ ]:
# Definimos la ruta de los archivos 

individual_clusters="%s.clusters" %(individual_footprint)
individual_labels="%s.labels" %(individual_footprint)

## Funciones

In [ ]:
def process_footprint(data,tests,log=False):
    from sklearn.cluster import MiniBatchKMeans
    #KMeans(init='k-means++', n_clusters=k, n_init=10)
    import datetime
    K={}
    for k in tests:
        if k<=len(data):
            if log:
                print("%s: processing %s"%(datetime.datetime.now(),k))
            K[k]=bench_k_means(MiniBatchKMeans(init='k-means++', n_clusters=k, batch_size=100,
                      n_init=10, max_no_improvement=10, verbose=0,
                      random_state=0),name="k-means++", data=data)
    return K

In [ ]:
def compute_best_k(x,y,occurrencies, plot=False,points=1000,sf=0.9):
    import numpy as np
    
    if len(x)<5:
        return max(1, round(np.sqrt(occurrencies/2)))
    
    from scipy.interpolate import interp1d
    from scipy.interpolate import UnivariateSpline
    spl = UnivariateSpline(x, y)
    spl.set_smoothing_factor(sf)
    xs = np.linspace(min(x), max(x), points)
    ys = spl(xs)
    idx_better_k=get_change_point(xs, ys)
    if plot:
        import pylab
        pylab.plot(xs,ys)
        
        pylab.scatter(xs[idx_better_k],ys[idx_better_k],s=20, marker='o')
        pylab.text(xs[idx_better_k],ys[idx_better_k],"bestK %s" %(np.round(xs[idx_better_k])))
        return int(np.round(xs[idx_better_k])),pylab
    return int(np.round(xs[idx_better_k]))

In [ ]:
def bench_k_means(estimator, name, data,distance_function=None):
    from sklearn import metrics
    from sklearn.metrics import silhouette_samples, silhouette_score
    import time
    t0 = time.time()
    if distance_function:
        estimator.fit(data,distance_function)
    else:
        estimator.fit(data)
    #cluster_labels = estimator.fit_predict(data)
    #silhouette_score_ = silhouette_score(data, cluster_labels)
    
    inertia=estimator.inertia_
    duration=time.time() - t0
    return {'inertia':inertia,'duration':duration, 'estimator':estimator}#,'silhouette':silhouette_score_}

def get_change_point(x, y):
    """
        Scelta del K migliore
        ::param x: lista di valori di K
        ::param y: lista di valori di SSE
    """
    import math
    max_d = -float('infinity')
    index = 0

    for i in range(0, len(x)):
        c = closest_point_on_segment(a=[x[0], y[0]], b=[x[len(x)-1], y[len(y)-1]], p=[x[i], y[i]])
        d = math.sqrt((c[0]-x[i])**2 + (c[1]-y[i])**2)
        if d > max_d:
            max_d = d
            index = i
    
    return index

def closest_point_on_segment(a, b, p):
    sx1 = a[0]
    sx2 = b[0]
    sy1 = a[1]
    sy2 = b[1]
    px = p[0]
    py = p[1]

    x_delta = sx2 - sx1
    y_delta = sy2 - sy1

    if x_delta == 0 and y_delta == 0:
        return p

    u = ((px - sx1) * x_delta + (py - sy1) * y_delta) / (x_delta * x_delta + y_delta *  y_delta)
    if u < 0:
        closest_point = a
    elif u > 1:
        closest_point = b
    else:
        cp_x = sx1 + u * x_delta
        cp_y = sy1 + u * y_delta
        closest_point = [cp_x, cp_y]

    return closest_point
	

## Individual Clustering

In [ ]:
# Numero de filas del archivo
f=open(individual_footprint)
num_rows = len(f.readlines())-1
f.close()

#<customer_id;year;week;profile_id;size;t1... tn >
import datetime
f=open(individual_footprint)
fw=open(individual_clusters,'w')  #uid,cluster_id,centroid
fw2=open(individual_labels,'w') #uid,year,week,cluster_id,profile

f.readline()
data=[] #buffer

footprints_clustered=0
footprints_clusters=0
n_cliente=0
contador = 0
temporal= 0
for row in f: #reading individual footprint
    row=row.strip().split(',') # leemos cada elemento da linea parseada por ","
    uid=row[0]
    year=row[1]
    week=row[2]
    size=int(row[5])
    profile=np.array([float(el) for el in row[6:]])
    # Individual clustering
    if uid==clientes[n_cliente]: # Para cada fila donde los "uid" son iguales 
        data.append(((uid,year,week),profile))     
        contador+=1
    else: #final de cliente
        
        #---------------------------------------------------------------------
        # procesar data
        #---------------------------------------------------------------------
        to_cluster=[el[1] for el in data]
        K=process_footprint(to_cluster,np.arange(1,len(to_cluster)))

        # choose k
        x=list(K.keys())
        y=[K[k]['inertia'] for k in K]
        best_k=compute_best_k(x,y,len(to_cluster))
        print('clustering: '+str(clientes[n_cliente])+' len data: '+str(len(data))+" best k: "+str(best_k))
        
        # clustering
        if best_k==1:
            #to few records
            cluster_centers_=[np.average(to_cluster,axis=0)]
            labels_=[0]*len(to_cluster)  
        else:
            cluster_centers_=K[best_k]['estimator'].cluster_centers_
            labels_=K[best_k]['estimator'].labels_
        
        #export individual centroids
        for i in np.arange(len(cluster_centers_)):
            string="%s;%s;%s\n"%(clientes[n_cliente],i,';'.join([str(el) for el in cluster_centers_[i]])) #uid,cluster_id,centroid
            fw.write(string)
            footprints_clusters+=1
        fw.flush()

        #export original data and labels
        for i in np.arange(len(data)):
            uid2=data[i][0]
            profile2=data[i][1]
            label2=labels_[i]
            string="%s;%s;%s;%s;%s\n" %(uid2[0],uid2[1],uid2[2],label2
                                                    ,';'.join([str(el) for el in profile2]))#uid,year,week,cluster_id,profile
            fw2.write(string)
            footprints_clustered+=1
        fw2.flush()
        #---------------------------------------------------------------------
        #---------------------------------------------------------------------
        
        

        data=[] #buffer
        data.append(((uid,year,week),profile))
        
        contador+=1
        temporal+=1
        n_cliente+=1
        
    if contador == num_rows:        # Para el ultimo cliente y ultima fila
        #---------------------------------------------------------------------
        # procesar data
        #---------------------------------------------------------------------
        to_cluster=[el[1] for el in data]
        K=process_footprint(to_cluster,np.arange(1,len(to_cluster)))

        # choose k
        x=list(K.keys())
        y=[K[k]['inertia'] for k in K]
        best_k=compute_best_k(x,y,len(to_cluster))
        print('clustering: '+str(clientes[n_cliente])+' len data: '+str(len(data))+" best k: "+str(best_k))
        
        # clustering
        if best_k==1:
            #to few records
            cluster_centers_=[np.average(to_cluster,axis=0)]
            labels_=[0]*len(to_cluster)  
        else:
            cluster_centers_=K[best_k]['estimator'].cluster_centers_
            labels_=K[best_k]['estimator'].labels_
        
        #export individual centroids
        for i in np.arange(len(cluster_centers_)):
            string="%s;%s;%s\n"%(uid,i,';'.join([str(el) for el in cluster_centers_[i]])) #uid,cluster_id,centroid
            fw.write(string)
            footprints_clusters+=1
        fw.flush()

        #export original data and labels
        for i in np.arange(len(data)):
            uid2=data[i][0]
            profile2=data[i][1]
            label2=labels_[i]
            string="%s;%s;%s;%s;%s\n" %(uid2[0],uid2[1],uid2[2],label2
                                                    ,';'.join([str(el) for el in profile2]))#uid,year,week,cluster_id,profile
            fw2.write(string)
            footprints_clustered+=1
        fw2.flush()
        #---------------------------------------------------------------------
        #---------------------------------------------------------------------
        
        print("final")   
    
    

temporal

In [ ]:
clientes

# Lista de clientes

In [ ]:
to_cluster=[el[1] for el in data]
K=process_footprint(to_cluster,np.arange(1,len(to_cluster)))

#choose k
x=K.keys()
x=list(x)
y=[K[k]['inertia'] for k in K]
best_k=compute_best_k(x,y,len(to_cluster))
print('clustering: '+str(uid)+' len data: '+str(len(data))+" best k: "+str(best_k))

In [ ]:
 #clustering
if best_k==1:
    #to few records
    cluster_centers_=[np.average(to_cluster,axis=0)]
    labels_=[0]*len(to_cluster)  
else:
    cluster_centers=K[best_k]['estimator'].cluster_centers_
    labels=K[best_k]['estimator'].labels_
            

In [ ]:
#export individual centroids
for i in np.arange(len(cluster_centers_)):
    string="%s;%s;%s\n"%(uid,i,';'.join([str(el) for el in cluster_centers_[i]])) #uid,cluster_id,centroid
    fw.write(string)
    footprints_clusters+=1
fw.flush()

#export original data and labels
for i in np.arange(len(data)):
    uid=data[i][0]
    profile=data[i][1]
    label=labels_[i]
    string="%s;%s;%s;%s;%s\n" %(uid[0],uid[1],uid[2],label
                                            ,';'.join([str(el) for el in profile]))#uid,year,week,cluster_id,profile
    fw2.write(string)
    footprints_clustered+=1
fw2.flush()

In [ ]:
cluster_centers

In [ ]:
y[11]

In [ ]:
K=process_footprint(to_cluster,np.arange(1,len(to_cluster)))

In [ ]:
len(to_cluster)

In [ ]:
K

In [ ]:
x=K.keys()
x=list(x)
len(x)

In [ ]:
y=[K[k]['inertia'] for k in K]
len(y)

In [ ]:
best_k=compute_best_k(x,y,len(to_cluster))
best_k

In [ ]:
from scipy.interpolate import UnivariateSpline
w = np.isnan(y)
y[w] = 0.0
spl = UnivariateSpline(x, y)

In [ ]:
# Plot the elbow
plt.plot(x, y, 'r-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()


In [ ]:
list(x)[0:18]


In [ ]:
y[0:18]

In [ ]:
spl = UnivariateSpline(list(x)[0:18], y[0:18])
spl

# Pruebas


In [ ]:
 # clustering dataset
# determine k using elbow method

from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt

x1 = np.array([3, 1, 1, 2, 1, 6, 6, 6, 5, 6, 7, 8, 9, 8, 9, 9, 8])
x2 = np.array([5, 4, 5, 6, 5, 8, 6, 7, 6, 7, 1, 2, 1, 2, 3, 2, 3])

plt.plot()
plt.xlim([0, 10])
plt.ylim([0, 10])
plt.title('Dataset')
plt.scatter(x1, x2)
plt.show()

# create new plot and data
plt.plot()
X = np.array(list(zip(x1, x2))).reshape(len(x1), 2)
colors = ['b', 'g', 'red']
markers = ['o', 'v', 's']

# k means determine k
distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(X)
    kmeanModel.fit(X)
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

 

In [ ]:
from __future__ import print_function

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

print(__doc__)

# Generating the sample data from make_blobs
# This particular setting has one distinct cluster and 3 clusters placed close
# together.
X, y = make_blobs(n_samples=500,
                  n_features=2,
                  centers=4,
                  cluster_std=1,
                  center_box=(-10.0, 10.0),
                  shuffle=True,
                  random_state=1)  # For reproducibility

range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

    plt.show()